In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
   precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef

In [2]:
data = pd.read_csv(r"C:\Users\Nilanjan\Desktop\paper\Revision\v1\SHP2_inhibitory_activity_prediction\10_fold_cross_validation\train_10folds_208.csv")
data = data.iloc[:,1:]
data

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IC50(microM),TARGET,Kfold
0,13.227657,-3.705912,13.227657,0.164190,0.626675,493.969,469.777,493.107434,176,0,...,0,0,0,0,0,0,0,8.2,0.0,8
1,13.143890,-0.107629,13.143890,0.025441,0.731324,450.996,423.780,450.160473,162,0,...,0,0,0,0,0,0,0,0.06,1.0,1
2,13.193346,-0.106925,13.193346,0.106925,0.787500,421.372,395.164,420.148367,150,0,...,0,0,0,0,0,0,0,19.0,0.0,5
3,13.349830,-3.633188,13.349830,0.144450,0.504377,516.450,493.266,515.083718,176,0,...,0,0,0,0,0,0,0,5.7,0.0,3
4,12.934251,-0.256713,12.934251,0.141991,0.833591,382.295,361.127,381.112316,134,0,...,0,0,0,0,0,0,0,0.105,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,12.580481,-0.654806,12.580481,0.000075,0.466149,450.619,412.315,450.277010,178,0,...,0,0,0,0,0,0,0,3.3,0.0,4
2735,11.795088,-1.575163,11.795088,0.003825,0.419983,382.372,364.228,382.116486,144,0,...,0,0,0,0,0,0,0,>50.0,0.0,8
2736,13.152750,-4.208578,13.152750,0.037250,0.698368,413.451,394.299,413.093308,150,0,...,0,0,0,0,0,0,0,181.0,0.0,3
2737,13.543157,-0.162092,13.543157,0.003333,0.500717,499.019,467.771,498.225850,186,0,...,0,0,0,0,0,0,0,0.035,1.0,2


In [3]:
data['TARGET'].value_counts()

0.0    1771
1.0     968
Name: TARGET, dtype: int64

In [4]:
data = data.drop(["IC50(microM)"],axis=1)

In [5]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.iloc[:,:-1].copy()
    labels = df.pop('TARGET')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(data))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [6]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)
    
    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [7]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    
    # create a layer that turns integer values into integer indices.
    index = layers.IntegerLookup(max_tokens=max_tokens)
    
    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)
    
    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    
    return lambda feature: encoder(index(feature))

In [8]:
def train_val_split(data, fold, batch_size):
    train = data[data.Kfold != fold].reset_index(drop=True)
    valid = data[data.Kfold == fold].reset_index(drop=True)
    train_ds = df_to_dataset(train, batch_size=batch_size)
    val_ds = df_to_dataset(valid, shuffle=False, batch_size=batch_size)
    
    return train_ds, val_ds, train, valid

In [9]:
def preprocess_dataset(numerical_features, categorical_features):
    all_inputs = []
    encoded_features = []

    # Numerical features.
    for header in numerical_features:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)
    
    hacc_col = tf.keras.Input(shape=(1,), name='NumHAcceptors', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='NumHAcceptors',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=20)
    encoded_hacc_col = encoding_layer(hacc_col)

    all_inputs.append(hacc_col)
    encoded_features.append(encoded_hacc_col)
    
    scc_col = tf.keras.Input(shape=(1,), name='NumSaturatedCarbocycles', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='NumSaturatedCarbocycles',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=5)
    encoded_scc_col = encoding_layer(scc_col)

    all_inputs.append(scc_col)
    encoded_features.append(encoded_scc_col)
    
    fbc_col = tf.keras.Input(shape=(1,), name='fr_bicyclic', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='fr_bicyclic',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=10)
    encoded_fbc_col = encoding_layer(fbc_col)
    all_inputs.append(fbc_col)
    encoded_features.append(encoded_fbc_col)
    
    return all_inputs, encoded_features

In [10]:
numerical_features = ['MaxPartialCharge', 'FpDensityMorgan2', 'BCUT2D_CHGLO', 'BCUT2D_MRHI',
       'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA3', 'SlogP_VSA3', 'SlogP_VSA8', 'EState_VSA6']

categorical_features = ['NumHAcceptors', 'NumSaturatedCarbocycles', 'fr_bicyclic']

In [11]:
class config:
    hidden_layer_1 = 16
    hidden_layer_2 = 64
    hidden_layer_3 = 128
    hidden_layer_4 = 256
    hidden_layer_5 = 32
    hidden_layer_6 = 8
    initializer = tf.keras.initializers.HeNormal()
    lr = 0.001
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    dropout = 0.1
    batch_size = 16
    epochs = 100

In [12]:
def AnnClassifier(all_inputs, encoded_features):
    all_features = tf.keras.layers.concatenate(encoded_features)
    x = tf.keras.layers.Dense(units=config.hidden_layer_1, activation="relu", 
                              kernel_initializer=config.initializer)(all_features)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_2, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_3, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_4, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_5, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_6, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(all_inputs, output)
    model.compile(optimizer = config.optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [13]:
def evaluate(fold, all_inputs, encoded_features, valid, val_ds):
    
    model = AnnClassifier(all_inputs, encoded_features)
    checkpoint = f".\\model_checkpoints\\fold_{fold}"
    model.load_weights(checkpoint)
    
    y_pred_proba = model.predict(val_ds)
    y_true = valid.TARGET.values
    auc = roc_auc_score(y_true, y_pred_proba)
    y_pred = (y_pred_proba > 0.5)
    
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC

In [14]:
final_features = numerical_features + categorical_features + ["TARGET","Kfold"]
data = data[final_features]
data

,MaxPartialCharge,FpDensityMorgan2,BCUT2D_CHGLO,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA3,SlogP_VSA3,SlogP_VSA8,EState_VSA6,NumHAcceptors,NumSaturatedCarbocycles,fr_bicyclic,TARGET,Kfold
0,0.332564,1.757576,-2.351100,7.887431,15.930471,17.667307,4.305216,26.033187,0.000000,18.199101,6,0,1,0.0,8
1,0.268453,2.066667,-2.414340,7.991393,5.948339,23.362825,14.535057,11.784535,0.000000,23.877982,9,0,0,1.0,1
2,0.262680,1.892857,-2.424946,6.426955,0.000000,35.689068,9.551078,0.000000,5.687386,22.766201,5,1,0,0.0,5
3,0.257838,1.500000,-2.351092,7.887435,10.023291,35.334614,4.305216,16.444113,0.000000,47.364479,4,0,1,0.0,3
4,0.264234,1.880000,-2.373477,6.433504,5.948339,35.334614,9.551078,7.047672,11.126903,25.246773,6,0,0,1.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734,0.308856,1.636364,-2.695144,6.061803,5.783245,45.420094,0.000000,9.589074,0.000000,6.076020,3,3,7,0.0,4
2735,0.394239,1.714286,-2.160783,6.360079,0.000000,11.840869,0.000000,14.325937,0.000000,31.127988,6,0,0,0.0,8
2736,0.335180,1.724138,-2.211061,7.931988,5.907180,0.000000,0.000000,27.659472,10.969244,18.199101,5,0,3,0.0,3
2737,0.265949,2.085714,-2.412877,6.355188,5.948339,11.600940,24.732420,11.784535,22.290781,23.877982,9,1,1,1.0,2


In [15]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []


for fold_ in range(10):
    train_ds, val_ds, train, valid = train_val_split(data, fold_, config.batch_size)
    all_inputs, encoded_features = preprocess_dataset(numerical_features, categorical_features)
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC = evaluate(fold_, all_inputs, encoded_features, valid, val_ds)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


Fold = 0, AUC = 0.9686622191011235, Accuracy = 0.9233576642335767,           Precision_1 = 0.9120879120879121, Precision_0 = 0.9289617486338798          Recall_1 = 0.8645833333333334, Recall_0 = 0.9550561797752809, F1Score = 0.8877005347593583, kappa = 0.8295918971746727, MCC = 0.8302755773923346


C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


Fold = 1, AUC = 0.9361931387966684, Accuracy = 0.8905109489051095,           Precision_1 = 0.8602150537634409, Precision_0 = 0.9060773480662984          Recall_1 = 0.8247422680412371, Recall_0 = 0.9265536723163842, F1Score = 0.8421052631578948, kappa = 0.7583632194720442, MCC = 0.7587571222872139


C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


...

Fold = 7, AUC = 0.9380278408759976, Accuracy = 0.9087591240875912,           Precision_1 = 0.8829787234042553, Precision_0 = 0.9222222222222223          Recall_1 = 0.8556701030927835, Recall_0 = 0.9378531073446328, F1Score = 0.8691099476439791, kappa = 0.799108452108628, MCC = 0.7993407530088565


C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


Fold = 8, AUC = 0.9584134195352089, Accuracy = 0.916058394160584,           Precision_1 = 0.9204545454545454, Precision_0 = 0.9139784946236559          Recall_1 = 0.8350515463917526, Recall_0 = 0.96045197740113, F1Score = 0.8756756756756756, kappa = 0.812540900707954, MCC = 0.8147357999691781


C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


Fold = 9, AUC = 0.9638064971751412, Accuracy = 0.9157509157509157,           Precision_1 = 0.8924731182795699, Precision_0 = 0.9277777777777778          Recall_1 = 0.8645833333333334, Recall_0 = 0.943502824858757, F1Score = 0.8783068783068784, kappa = 0.8139059304703476, MCC = 0.8141458071798944


Mean Scores: AUC = 0.9561567673699528,       Accuracy = 0.9120130477794712,       Precision_1 = 0.8950362157275453, Precision_0 = 0.9209027187158604      Recall_1 = 0.851267182130584, Recall_0 = 0.945223132101822      F1Score = 0.8723929057587572       Kappa = 0.805327512400323       MCC = 0.8061316803948759


In [18]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.923358,0.968662,0.912088,0.928962,0.864583,0.955056,0.887701,0.829592,0.830276
1,0.890511,0.936193,0.860215,0.906077,0.824742,0.926554,0.842105,0.758363,0.758757
2,0.901460,0.949764,0.872340,0.916667,0.845361,0.932203,0.858639,0.783037,0.783265
3,0.919708,0.969771,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
4,0.934307,0.976527,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
5,0.897810,0.937853,0.870968,0.911602,0.835052,0.932203,0.852632,0.774472,0.774875
6,0.912409,0.962549,0.919540,0.909091,0.824742,0.960452,0.869565,0.803924,0.806620
7,0.908759,0.938028,0.882979,0.922222,0.855670,0.937853,0.869110,0.799108,0.799341
8,0.916058,0.958413,0.920455,0.913978,0.835052,0.960452,0.875676,0.812541,0.814736
9,0.915751,0.963806,0.892473,0.927778,0.864583,0.943503,0.878307,0.813906,0.814146


In [19]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.923358,0.968662,0.912088,0.928962,0.864583,0.955056,0.887701,0.829592,0.830276
1,0.890511,0.936193,0.860215,0.906077,0.824742,0.926554,0.842105,0.758363,0.758757
2,0.901460,0.949764,0.872340,0.916667,0.845361,0.932203,0.858639,0.783037,0.783265
3,0.919708,0.969771,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
4,0.934307,0.976527,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
5,0.897810,0.937853,0.870968,0.911602,0.835052,0.932203,0.852632,0.774472,0.774875
6,0.912409,0.962549,0.919540,0.909091,0.824742,0.960452,0.869565,0.803924,0.806620
7,0.908759,0.938028,0.882979,0.922222,0.855670,0.937853,0.869110,0.799108,0.799341
8,0.916058,0.958413,0.920455,0.913978,0.835052,0.960452,0.875676,0.812541,0.814736
9,0.915751,0.963806,0.892473,0.927778,0.864583,0.943503,0.878307,0.813906,0.814146


In [20]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
0,0.923358,0.968662,0.912088,0.928962,0.864583,0.955056,0.887701,0.829592,0.830276
1,0.890511,0.936193,0.860215,0.906077,0.824742,0.926554,0.842105,0.758363,0.758757
2,0.901460,0.949764,0.872340,0.916667,0.845361,0.932203,0.858639,0.783037,0.783265
3,0.919708,0.969771,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
4,0.934307,0.976527,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
5,0.897810,0.937853,0.870968,0.911602,0.835052,0.932203,0.852632,0.774472,0.774875
6,0.912409,0.962549,0.919540,0.909091,0.824742,0.960452,0.869565,0.803924,0.806620
7,0.908759,0.938028,0.882979,0.922222,0.855670,0.937853,0.869110,0.799108,0.799341
8,0.916058,0.958413,0.920455,0.913978,0.835052,0.960452,0.875676,0.812541,0.814736
9,0.915751,0.963806,0.892473,0.927778,0.864583,0.943503,0.878307,0.813906,0.814146


In [21]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

,Accuracy,AUC,Precision_1,Precision_0,Recall_1,Recall_0,F1score,Kappa,MCC
Fold_0,0.923358,0.968662,0.912088,0.928962,0.864583,0.955056,0.887701,0.829592,0.830276
Fold_1,0.890511,0.936193,0.860215,0.906077,0.824742,0.926554,0.842105,0.758363,0.758757
Fold_2,0.901460,0.949764,0.872340,0.916667,0.845361,0.932203,0.858639,0.783037,0.783265
Fold_3,0.919708,0.969771,0.912088,0.923497,0.855670,0.954802,0.882979,0.821962,0.822933
Fold_4,0.934307,0.976527,0.907216,0.949153,0.907216,0.949153,0.907216,0.856369,0.856369
Fold_5,0.897810,0.937853,0.870968,0.911602,0.835052,0.932203,0.852632,0.774472,0.774875
Fold_6,0.912409,0.962549,0.919540,0.909091,0.824742,0.960452,0.869565,0.803924,0.806620
Fold_7,0.908759,0.938028,0.882979,0.922222,0.855670,0.937853,0.869110,0.799108,0.799341
Fold_8,0.916058,0.958413,0.920455,0.913978,0.835052,0.960452,0.875676,0.812541,0.814736
Fold_9,0.915751,0.963806,0.892473,0.927778,0.864583,0.943503,0.878307,0.813906,0.814146


# Bayesian optimization

In [22]:
from hyperopt import hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [23]:
def AnnClassifier_(all_inputs, encoded_features,space):
    all_features = tf.keras.layers.concatenate(encoded_features)
    x = tf.keras.layers.Dense(units=config.hidden_layer_1, activation="relu", 
                              kernel_initializer=config.initializer)(all_features)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_2, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_3, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_4, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_5, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_6, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(all_inputs, output)
    config.lr = space["learning_rate"]
    model.compile(optimizer = config.optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [31]:
class BayesianOPT:
    def __init__(self, data, space):
        self.space = space
        self.data = data
        self.trials = Trials()
    
    def _objective(self,space):
        model = AnnClassifier_(all_inputs, encoded_features, space)
        accuracies = []
        for fold_ in range(10):
            train_ds, val_ds, train, valid = train_val_split(data, fold_, config.batch_size)
            
            # evaluation = [( X_train, y_train), ( X_test, y_test)]
        

            checkpoint = f".\\model_checkpoints\\fold_{fold_}"
            model.load_weights(checkpoint)


            pred = model.predict(val_ds)
            y_test = valid.TARGET.values
            accuracy = accuracy_score(y_test, pred>0.5)
            accuracies.append(accuracy)
        
        final_accuracy = np.mean(np.array(accuracies))
        #print ("SCORE:", final_accuracy)
        return {'loss': -final_accuracy, 'status': STATUS_OK }
    
    def search_hyperparameters(self):

        best_hyperparams = fmin(fn = self._objective,
                                space = self.space,
                                algo = tpe.suggest,
                                max_evals = 500,
                                trials = self.trials)
        return best_hyperparams

In [32]:
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'epochs': hp.choice('epochs', [10, 20, 30, 40, 50]),
    'batch_size': hp.choice('batch_size', [16, 32, 64, 128]),
}

In [33]:
bayes_opt = BayesianOPT(data, space)
best_params = bayes_opt.search_hyperparameters()
print(best_params)

  0%|                                                                          | 0/500 [00:00<?, ?trial/s, best loss=?]

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi

  0%|                                               | 1/500 [00:03<27:32,  3.31s/trial, best loss: -0.8890203470495441]

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi

  0%|▏                                              | 2/500 [00:06<27:18,  3.29s/trial, best loss: -0.8890203470495441]

...

100%|████████████████████████████████████████████▊| 498/500 [26:57<00:07,  3.87s/trial, best loss: -0.8890203470495441]

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi

100%|████████████████████████████████████████████▉| 499/500 [27:00<00:03,  3.86s/trial, best loss: -0.8890203470495441]

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi

100%|█████████████████████████████████████████████| 500/500 [27:04<00:00,  3.25s/trial, best loss: -0.8890203470495441]
{'batch_size': 0, 'epochs': 0, 'learning_rate': 0.016587132568474387}


In [35]:
best_params["epochs"] = 10
best_params["batch_size"] = 16

# Train the model with optimized hyperparameters

In [36]:
def train(fold, all_inputs, encoded_features, train_ds, val_ds, epochs):
    
    print(f"Fold : {fold}")
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    
    logdir = f'.\\tensorboard_logs\\scalars\\fold_{fold}\\'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,write_graph=True,update_freq=1)
    
    checkpoint_filepath = f'.\\model_checkpoints\\fold_{fold}\\'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    model = AnnClassifier_(all_inputs, encoded_features, best_params)

    model.fit(train_ds, 
            epochs = epochs, 
            validation_data = val_ds,
            callbacks=[early_stop, tensorboard_callback, model_checkpoint_callback])
    
    # model_loss = pd.DataFrame(log.history.history)
    # model_loss.to_csv(f".\\Metrics_\\metrics_{fold}.csv", index=False)

In [37]:
def my_lambda(x):
    return tf.cast(x, tf.float32)

In [55]:
def run(fold, data, best_params):
    train_ds, val_ds, train_, valid = train_val_split(data, fold_, best_params["batch_size"])
    # initialize Logistic Regression model
    # train_ds = train_ds.map(my_lambda)
    # val_ds = val_ds.map(my_lambda)
    all_inputs, encoded_features = preprocess_dataset(numerical_features, categorical_features)
    train(fold, all_inputs, encoded_features, train_ds, val_ds, best_params["epochs"])
    model = AnnClassifier_(all_inputs, encoded_features, best_params)
    checkpoint = f".\\model_checkpoints\\fold_{fold}"
    model.load_weights(checkpoint)
    y_pred = model.predict(val_ds)
    y_pred = y_pred > 0.5
    y_true = valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    auc = roc_auc_score(y_true, y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [56]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data, best_params)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

C:\Users\Nilanjan\AppData\Local\Temp\ipykernel_25296\1507979022.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}


Fold : 0
Epoch 1/10


C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


155/155 [==============================] - 6s 20ms/step - loss: 0.4379 - accuracy: 0.8130 - val_loss: 0.4415 - val_accuracy: 0.8467


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_0\assets


...

INFO:tensorflow:Assets written to: .\model_checkpoints\fold_9\assets


INFO:tensorflow:Assets written to: .\model_checkpoints\fold_9\assets


Epoch 6/10
155/155 [==============================] - 3s 17ms/step - loss: 0.3228 - accuracy: 0.8690 - val_loss: 0.2412 - val_accuracy: 0.8974
Epoch 7/10
155/155 [==============================] - 3s 17ms/step - loss: 0.2939 - accuracy: 0.8804 - val_loss: 0.2243 - val_accuracy: 0.8938
Epoch 8/10
155/155 [==============================] - 3s 16ms/step - loss: 0.3040 - accuracy: 0.8788 - val_loss: 0.2329 - val_accuracy: 0.8974
Epoch 9/10
155/155 [==============================] - 2s 15ms/step - loss: 0.3017 - accuracy: 0.8771 - val_loss: 0.2159 - val_accuracy: 0.9084
Epoch 10/10
155/155 [==============================] - 2s 15ms/step - loss: 0.2705 - accuracy: 0.8865 - val_loss: 0.2150 - val_accuracy: 0.9158


C:\Users\Nilanjan\Envs\tensorflow_2.6\lib\site-packages\keras\engine\functional.py:582: UserWarning: Input dict contained keys ['TARGET'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


Fold = 9, AUC = 0.9101341807909604, Accuracy = 0.9267399267399268,           Precision_1 = 0.9318181818181818, Precision_0 = 0.9243243243243243          Recall_1 = 0.8541666666666666, Recall_0 = 0.9661016949152542, F1Score = 0.8913043478260869, kappa = 0.8362131029517639, MCC = 0.8380134908187058


Mean Scores: AUC = 0.8796404616270113,       Accuracy = 0.8970535547177884,       Precision_1 = 0.880537677073284, Precision_0 = 0.9055820885727318      Recall_1 = 0.8202534364261167, Recall_0 = 0.9390274868279057      F1Score = 0.8489448088100655       Kappa = 0.7710508536166462       MCC = 0.7725417423486571
